# Validation by Schema

[python-jsonschema](https://python-jsonschema.readthedocs.io/en/latest/) is the library to ingest the associated JSON Schema at runtime and validate inputs.

In [1]:
schema = {
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "$id": "https://eoap.github.io/cwl2ogc/array-inputs/inputs.yaml",
  "description": "The schema to represent a array-inputs inputs definition",
  "type": "object",
  "required": [
    "filesA",
    "filesB",
    "filesC"
  ],
  "properties": {
    "filesA": {
      "$ref": "#/$defs/filesA"
    },
    "filesB": {
      "$ref": "#/$defs/filesB"
    },
    "filesC": {
      "$ref": "#/$defs/filesC"
    }
  },
  "additionalProperties": False,
  "$defs": {
    "filesA": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesB": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "filesC": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}

## 1. Parse the JSON Schema

In [2]:
from assertions_mate.jsonschema_validator import JSONSchemaValidator
from assertions_mate.error_models import (
    ErrorDetail,
    ServerError
)
from jsonschema.exceptions import SchemaError
import json
import sys

validator = None
try:
    validator = JSONSchemaValidator(schema)
except SchemaError as schema_error:
    print(
        json.dumps(
            ServerError(errors=[ErrorDetail(detail=schema_error.message)]),
            indent=2
        )
    )

## 2. Define your inputs set

In [3]:
inputs = {
    "aoi": "-118.985,38.432,-118.183,38.938",
    "filesB": "EPSG:4326",
    "bands": [ "green", "nir08" ],
    "item": {
        "class":"https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL",
        "value":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1"
    }
}

## 3. Validate the inputs and report the errors

In [4]:
if validator:
    error = validator.validate_inputs(inputs)

    if error:
        print(error.model_dump_json(indent=2, exclude_none=True))
    else:
        # No violations — choose what to output. A simple success marker is fine:
        print("Go ahead on submitting the request to the next level validation")

{'message': "'filesA' is a required property", 'path': deque([]), 'relative_path': deque([]), 'schema_path': deque(['required']), 'relative_schema_path': deque(['required']), 'context': [], 'cause': None, 'validator': 'required', 'validator_value': ['filesA', 'filesB', 'filesC'], 'instance': {'aoi': '-118.985,38.432,-118.183,38.938', 'filesB': 'EPSG:4326', 'bands': ['green', 'nir08'], 'item': {'class': 'https://raw.githubusercontent.com/eoap/schemas/main/url.yaml#URL', 'value': 'https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC08_L2SP_042033_20231007_02_T1'}}, 'schema': {'$schema': 'https://json-schema.org/draft/2020-12/schema', '$id': 'https://eoap.github.io/cwl2ogc/array-inputs/inputs.yaml', 'description': 'The schema to represent a array-inputs inputs definition', 'type': 'object', 'required': ['filesA', 'filesB', 'filesC'], 'properties': {'filesA': {'$ref': '#/$defs/filesA'}, 'filesB': {'$ref': '#/$defs/filesB'}, 'filesC': {'$ref': '#/$defs/file